In [1]:
import pandas, sqlalchemy
from datetime import datetime

In [2]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

In [3]:
Base = automap_base()

In [4]:
engine = create_engine('postgres+psycopg2://postgres@localhost:5436/anhperf_dev')

/Users/likit/.virtualenvs/anherf/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [5]:
Base.prepare(engine, reflect=True)

In [6]:
Base.classes.keys()

['alembic_version', 'roles', 'participants', 'registrations', 'checkins']

In [7]:
session = Session(engine)

In [8]:
Role = Base.classes['roles']

In [9]:
Participant = Base.classes['participants']

In [10]:
Registration = Base.classes['registrations']

In [11]:
speaker_role = Role(desc='speaker')
guest_role = Role(desc='guest')
vip_role = Role(desc='vip')
participant_role = Role(desc='participant')
invitee_role = Role(desc='invitee')

In [12]:
committee_role = Role(desc='committee')
session.add(committee_role)

In [13]:
session.add(Role(desc='rapporteur'))

In [14]:
session.add(speaker_role)
session.add(guest_role)
session.add(vip_role)
session.add(participant_role)
session.add(invitee_role)
session.commit()

In [15]:
def add_new_participant(role, data):
    for item in data.iterrows():
        row = item[1]
        if not pandas.isna(row['user_email']):
            email = row['user_email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname)
                continue
        else:
            email = ''
        new_par = Participant(title=row['name_title'],
                         firstname=row['first_name'],
                         lastname=row['last_name'],
                         email=email,
                         faculty=row['faculty'],
                         affil=row['university'],
                         mobile=row['mobile'],
                         delivery_address=row['delivery_address'],
                         position_type=row['position_type'],
                        )
        new_par.roles = role # reflection automatically add s to the role attribute
        registered_at = None if pandas.isna(row['user_registered']) else row['user_registered']
        pay_status = not pandas.isna(row['paid_at'])
        new_par.registrations_collection.append(Registration(
                                                    registered_at=registered_at,
                                                    pay_status=pay_status,
                                                    payment_required=True)
                                               )
        session.add(new_par)
    session.commit()

In [16]:
def add_new_committee(role, data):
    for item in data.iterrows():
        row = item[1]
        year = 2018
        if not pandas.isna(row['user_email']):
            email = row['user_email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        new_par = Participant(title=row['name_title'],
                         firstname=row['first_name'],
                         lastname=row['last_name'],
                         email=email,
                         faculty=row['faculty'],
                         affil=row['university'],
                         mobile=row['mobile'],
                         delivery_address=row['delivery_address'],
                         position_type=row['position_type'])
        new_par.roles = role
        registered_at = None if pandas.isna(row['user_registered']) else row['user_registered']
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [17]:
def add_new_invitee(role, data):
    for idx,row in data.iterrows():
        year = 2018
        #regcode = '{}{:05}'.format(year, row['ID'])
        if not pandas.isna(row['email']):
            email = row['email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        new_par = Participant(old_id=None,
                        title=row['title'],
                         firstname=row['firstname'],
                         lastname=row['lastname'],
                         email=email,
                         faculty=row['faculty'],
                         affil=row['affiliation'],
                         mobile=row['tel'],
                         delivery_address=row['address'],
                         position_type=row['position']
                        )
        new_par.roles = role
        registered_at = row['datetime']
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [18]:
def add_new_rap(role, data):
    for idx,row in data.iterrows():
        year = 2018
        if not pandas.isna(row['email']):
            email = row['email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        #regcode = '{}{:05}'.format(year, row['ID'])
        new_par = Participant(old_id=None,
                        title=row['title'],
                         firstname=row['firstname'],
                         lastname=row['lastname'],
                         email=email,
                         affil=row['university'],
                         mobile=row['mobile'],
                         position_type=row['position']
                        )
        new_par.roles = role
        registered_at = row['registered']
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [19]:
def add_new_speakers(role, data):
    for item in data.iterrows():
        row = item[1]
        year = 2018
        if pandas.isna(row['user_email']):
            email = row['user_email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        new_par = Participant(title=row['name_title'],
                         firstname=row['first_name'],
                         lastname=row['last_name'],
                         email=row['user_email'].strip(),
                         faculty=row['faculty'],
                         affil=row['university'],
                         mobile=row['mobile'],
                         delivery_address=row['delivery_address'],
                         position_type=row['position_type'])
        new_par.roles = role
        registered_at = None if pandas.isna(row['user_registered']) else row['user_registered']
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [20]:
def add_new_speakers_2(role, data):
    for item in data.iterrows():
        row = item[1]
        year = 2018
        if pandas.isna(row['email']):
            email = row['email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        new_par = Participant(firstname=row['firstname'],
                         lastname=row['lastname'],
                         email=row['email'].strip(),
                         faculty=row['faculty'],
                         affil=row['university'],
                         mobile=row['mobile'],
                         position_type=row['position'])
        new_par.roles = role
        registered_at = None if pandas.isna(row['registered']) else row['registered']
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [21]:
def add_new_committee_network(role, data):
    for item in data.iterrows():
        row = item[1]
        year = 2018
        if not pandas.isna(row['email']):
            email = row['email'].strip().lower()
            existing_participant = session.query(Participant).filter(Participant.email==email).first()
            if existing_participant:
                print(existing_participant.firstname, existing_participant.lastname)
                continue
        else:
            email = ''
        
        new_par = Participant(title=row['title'],
                         firstname=row['firstname'],
                         lastname=row['lastname'],
                         email=email,
                         affil=row['affiliation'],
                         mobile=row['mobile'])
        new_par.roles = role
        registered_at = None if pandas.isna(row['registered']) else row['registered']
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                             pay_status=False,
                                                             payment_required=False))
        session.add(new_par)
    session.commit()

In [22]:
participant_data = pandas.read_excel('register 06.11.2018.xlsx', sheet_name='participants')
participant_role = session.query(Role).filter(Role.desc=='participant').first()
add_new_participant(participant_role, participant_data)

วิวรรธน์


In [23]:
rap_data = pandas.read_excel('register 06.11.2018.xlsx', sheet_name='rapporteurs')
rapporteur_role = session.query(Role).filter(Role.desc=='rapporteur').first()
add_new_rap(rapporteur_role, rap_data)

อภิรดี เจริญนุกูล


In [24]:
committee_data = pandas.read_excel('register 06.11.2018.xlsx', sheet_name='committee')
committee_role = session.query(Role).filter(Role.desc=='committee').first()
add_new_committee(committee_role, committee_data)

ศุกร์ใจ เจริญสุช
ธนชัย พนาพุฒิ
วัชราภรณ์ เชี่ยววัฒนา
แสงเทียน ธรรมลิขิตกุล
พีรนันท์ วิศาลสกุลวงษ์
ภาวิดา พุทธิขันธ์
บุหงา ตโนภาส
รัตนา จารุวรรโณ
อังคณา สุเมธสิทธิกุล


In [25]:
invitee_data = pandas.read_excel('invitee_04112018.xlsx')
invitee_role = session.query(Role).filter(Role.desc=='invitee').first()
add_new_invitee(invitee_role, invitee_data)

ธานี กล่อมใจ
ขวัญตา บุญวาศ
ดร.ศรีสุดา วงศ์วิเศษกุล
วรุณยุพา รอยกุลเจริญ
นวลใย พิศชาติ
สมใจ ลือวิเศษไพบูลย์
พิมพ์เพชร สุขุมาลไพบูลย์
ประไพพิศ สิงหเสม
จิราพร วัฒนศรีสิน
วรรณภา ประทุมโทน
นงเยาว์ สมพิทยานุรักษ์
สุวิมล ล้วนเส้ง
วิชิต โนสูงเนิน
ประยุทธ ภูวรัตนาวิวิธ
สุวัฒนา จุฬาวัฒนทล
ชุตินันท์ ประสิทธิ์ภูริปรีชา
นุชนาถ วิชิต
รัชนี สรรเสริญ
อารยะ ไข่มุกด์
มัณฑนา เหมชะญาติ
ดวงพร หุ่นตระกูล
วิรัตน์ นิวัฒนนันท์
อรทิพา   ส่องศิริ
นิตยา ศรีสุข
เทพลักษ์ ศิริธนะวุฒิชัย
สุภาเพ็ญ ปาณะวัฒนพิสุทธิ์
ประดิษฐ์   ปฐวีศรีสุธา


In [26]:
speakers_data = pandas.read_excel('register 06.11.2018.xlsx', 'speakers')
speaker_role = session.query(Role).filter(Role.desc=='speaker').first()
add_new_speakers(speaker_role, speakers_data)

In [27]:
speakers_data_2 = pandas.read_excel('register 06.11.2018.xlsx', 'Sheet1')
add_new_speakers_2(speaker_role, speakers_data_2)

In [28]:
network_data = pandas.read_excel('register 06.11.2018.xlsx', sheet_name='network')
committee_role = session.query(Role).filter(Role.desc=='committee').first()
add_new_committee_network(committee_role, network_data)

siriwat Tiptaradol


In [38]:
notitles = []
for participant in session.query(Participant):
    if participant.title == 'NaN':
        notitles.append(participant)

In [42]:
for p in notitles:
    p.title='คุณ'
    print(p.title, p.firstname, p.lastname)

คุณ ศิริตรี สุทธจิตต์
คุณ วัชราภรณ์ เชี่ยววัฒนา
คุณ กฤษณา  ศิริวิบุลยกิติ
คุณ พัชริี  ดำรงสุนทรชัย
คุณ จารุภา จิรโสภณ


In [44]:
for p in notitles:
    session.add(p)
session.commit()

In [58]:
p = session.query(Participant).filter(Participant.id==546).first()
email1, email2 = p.email.split(', \n')
p.email = email2
session.add(p)
session.commit()

In [60]:
p = session.query(Participant).filter(Participant.lastname=='ดำรงสุนทรชัย').first()

In [66]:
p.firstname = 'พัชรี'
session.add(p)
session.commit()

In [67]:
p.firstname

'พัชรี'

In [64]:
p.email = 'bow_bow999@hotmail.com'
session.add(p)
session.commit()

In [65]:
p.id

205